In [15]:
import requests
import pandas as pd
import csv
import json
import matplotlib.pyplot as plt
import numpy as np

#pd.options.mode.chained_assignment = None  # default='warn'

url = "https://api.census.gov/data/2019/abscs"

api_key = "?"

fetchedData = requests.get( url + "?get=GEO_ID,FIRMPDEMP,EMPSZFI,EMPSZFI_LABEL,EMP,PAYANN,NAME,RCPPDEMP&for=us:*&key=" + api_key)

df = pd.DataFrame(fetchedData.json())

df.columns = df.iloc[0]

df = df.drop(df.index[0])

df


,GEO_ID,FIRMPDEMP,EMPSZFI,EMPSZFI_LABEL,EMP,PAYANN,NAME,RCPPDEMP,us
1,0100000US,5771292,001,All firms,128898226,7227585564,United States,40073253698,1
2,0100000US,730078,611,Firms with no employees,0,50074562,United States,298164421,1
3,0100000US,2826849,612,Firms with 1 to 4 employees,5632339,233431702,United States,1266736429,1
4,0100000US,959641,620,Firms with 5 to 9 employees,6329828,257601290,United States,1142368704,1
5,0100000US,609335,630,Firms with 10 to 19 employees,8211284,347772825,United States,1440258482,1
6,0100000US,406468,641,Firms with 20 to 49 employees,12230321,540126965,United States,2445916742,1
7,0100000US,126504,642,Firms with 50 to 99 employees,8663459,420744459,United States,2187018333,1
8,0100000US,70299,651,Firms with 100 to 249 employees,10646783,581208064,United States,3210325608,1
9,0100000US,21860,652,Firms with 250 to 499 employees,7588844,427858336,United States,2562327727,1
10,0100000US,20256,657,Firms with 500 employees or more,69595369,4368767359,United States,25520137251,1


In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

df_ploty = df.drop([1])

df_ploty["PAYANN"] = df_ploty["PAYANN"].astype(np.int64)
df_ploty["FIRMPDEMP"] = df_ploty["FIRMPDEMP"].astype(np.int64)
df_ploty["EMP"] = df_ploty["EMP"].astype(np.int64)
df_ploty["RCPPDEMP"] = df_ploty["RCPPDEMP"].astype(np.int64)


df_ploty = df_ploty.rename(columns={"PAYANN": "Annual payroll","FIRMPDEMP": "Number of employer firms","EMP": "Number of employees","EMPSZFI_LABEL": "Firm Size","RCPPDEMP": "revenue of employer firms"})

app = dash.Dash(__name__)

app.layout = html.Div([
    html.P("Names:"),
    dcc.Dropdown(
        id='names',
        options=[{'value': x, 'label': x} 
                 for x in ['Number of employer firms', 'Number of employees', 'Annual payroll',"revenue of employer firms"]],
        clearable=False
    ),
    html.P("Types:"),
    dcc.Dropdown(
        id = "types",
        options=[{"value" : x, "label": x} for x in ["bar","pie"]],
        clearable=False
    ),
    dcc.Graph(id="pie-chart"),
])

@app.callback(
    Output("pie-chart", "figure"),
    [Input("names", "value"),
     Input("types", "value")])
     
def generate_chart(values,type):
    if type == "pie":
        fig = px.pie(df_ploty,values =values,names = "Firm Size")
    elif type == "bar":
        fig = px.bar(df_ploty,x = "Firm Size",y = values,text_auto = ".3s")
    return fig

app.run_server()